In [ ]:
from struct_dbn import dynamicBN
import scipy.stats as stats
import numpy as np
import math

In [ ]:
# config = {"T": *,
#           "ncycles": *,
#           "d0_mean": *,
#           "dcrit": *,
#           "q_mean": *,
#           "q_cov": *,
#           "time_variant_q": False, 
#           "epsilonq_std": None}

config = {"T": 20,
          "ncycles": 3.5e7,
          "d0_mean": 0.1235,
          "dcrit": 16,
          "q_mean": 6.4839,
          "q_cov": 0.2,
          "time_variant_q": True, 
          "epsilonq_std": 0.3242}
struct = dynamicBN(config)
print(struct.time_variant_q)
print(struct.epsilonq_std)

In [ ]:
# cg_params = {"lnC_mean": *,
#              "lnC_std": *,
#              "m": *,
#              "Ynormal_mean": *,
#              "Ynormal_cov": *,
#              "nsamples": *}

cg_params = {"lnC_mean": -27.7903,
             "lnC_std": 0.3473,
             "m": 3,
             "Ynormal_mean": 1,
             "Ynormal_cov": 0.1,
             "nsamples": 1e6}
struct.crack_growth(cg_params)

In [ ]:
n_dstates= 30
n_qstates = 20
struct.transition_models(n_dstates, n_qstates)

In [ ]:
struct.Tr

In [ ]:
belief0_eps = np.array([0.1, 0.25])
# epsilon_twin = np.random.beta(2,10,(1000000,1))
# eps_interv = np.arange(11)*0.1
# import matplotlib.pyplot as plt
# plt.hist(epsilon_twin,epsilon_interv)
# belief0_eps, _ = np.histogram(epsilon_twin, eps_interv)
# belief0_eps = belief0_eps/np.sum(belief0_eps)

n_twinstates = 3
belief0_twin = np.zeros(n_twinstates)
belief0_twin[-1] = 1
# No-nothing (0, 1)
T0_twin = np.zeros((n_twinstates, n_twinstates))
T0_twin[0,:] = np.array([0, 1, 0])
T0_twin[1,:] = np.array([0, 1, 0])
T0_twin[2,:] = np.array([0, 0, 1])

# Repair component without install sensor (4)
Tr_twin = np.zeros((n_twinstates, n_twinstates))
Tr_twin[0,:] = np.array([0, 0, 1])
Tr_twin[1,:] = np.array([0, 0, 1])
Tr_twin[2,:] = np.array([0, 0, 1])

# (Re)install sensor  (2, 3, 5)
Ts_twin = np.zeros((n_twinstates, n_twinstates))
Ts_twin[0,:] = np.array([1, 0, 0])
Ts_twin[1,:] = np.array([1, 0, 0])
Ts_twin[2,:] = np.array([1, 0, 0])
T0_twin, Tr_twin, Ts_twin

In [ ]:
# obs_params = {"inspect": False, 
#               "beta0": 7.3704, "beta1": 2.092, "sigma_epsilon": 4.189, "det_thres": 5.4898,
#               "monitor": False,
#               "error_std": None}

obs_params = {"inspect": True,
              "monitor": True,
              "error_cov": 0.07}
struct.observation_models(obs_params)


In [ ]:
np.savez_compressed('Dr203020_Tw03', q_interv=struct.q_interv, d_interv=struct.d_interv,
         belief0=struct.b0, belief0_twin=belief0_twin, belief0_eps=belief0_eps,
         T0=struct.T0, Tr=struct.Tr, 
         T0_twin=T0_twin, Tr_twin=Tr_twin, Ts_twin=Ts_twin, 
         O_ins=struct.O['ins'], O_monitor=struct.O['monitor'], O_ins_monitor=struct.O['ins_monitor'])